In [1]:
# Cell 1 - imports e instalación (ejecutar si hace falta)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
sns.set(style="darkgrid")

In [2]:
array_data = []

for i in range(1,6):
    df = pd.read_csv(f"dataset_wear_{i}.csv")
    # convertir timestamp a datetime si está en ISO
    if "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
    # ordenar por tiempo
    df = df.sort_values(["device_id", "timestamp"])
    df.reset_index(drop=True, inplace=True)
    df.head()

    WINDOW_MINUTES = 3
    WINDOW = pd.Timedelta(minutes=WINDOW_MINUTES)
    start_ts = df["timestamp"].min()
    end_global = df["timestamp"].max()

    while start_ts + WINDOW <= end_global:
        #Obtencion registros entre el tiempo especifico
        end_ts = start_ts + WINDOW
        data_time = df[(df["timestamp"] > start_ts) & (df["timestamp"] <= end_ts)]
        #Contruccion de dataset con los 10 primeros vehiculos
        for veh_i, window in data_time.groupby("device_id"):
            try:
                veh_num = int(veh_i.replace("veh_", ""))
            except:
                continue
            if veh_num > 10:
                continue
            window = window[window["speed"] > 0]
            x = np.linspace(0, 1, len(window))
            dic = {
                "speed_mean": window["speed"].mean(),
                "speed_std": window["speed"].std(),
                #"speed_trend": np.polyfit(x, window["speed"], 1)[0],
                "fuel_mean": window["fuel_consumption"].mean(),
                "fuel_std": window["fuel_consumption"].std(),
                "temp_mean": window["engine_temperature"].mean(),
                "temp_max": window["engine_temperature"].max(),
                #"temp_trend": np.polyfit(x, window["engine_temperature"], 1)[0],
                "Maintenance": int(window["Maintenance"].mean() >= 0.5)
            }
            array_data.append(dic)
        start_ts = end_ts

In [3]:
#Conversion de array a dataframe
df_ml = pd.DataFrame(array_data)
print(df_ml.head())
print(df_ml.shape)

   speed_mean  speed_std  fuel_mean  fuel_std  temp_mean  temp_max  \
0   35.247143  18.106448   3.190000  1.990609  87.624286    101.07   
1   51.551176  19.602963   4.645294  2.929431  95.776471    116.43   
2   32.182308  23.743977   3.322308  2.747863  86.092308    111.96   
3   48.806154  26.267518   4.930000  4.060322  94.401538    118.96   
4   23.395714  14.581911   2.060000  1.641135  81.697857     98.80   

   Maintenance  
0            0  
1            0  
2            0  
3            0  
4            0  
(690, 7)


In [4]:
#Reemplazar valores nulos
df_clean = df_ml.copy()
for col in df_clean.columns:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())


In [8]:
path="dataset_vehicle_wear.csv"
df_clean.to_csv(path, index=False)
print(f"CSV guardado en: {os.path.abspath(path)} ({len(df)} filas).")

CSV guardado en: b:\Documentos\Universidad-Materias\Administrador de Servidores\Proyecto\sistema_Gestion_Flotas\dataset_vehicle_wear.csv (5729 filas).


In [9]:
df_clean

,speed_mean,speed_std,fuel_mean,fuel_std,temp_mean,temp_max,Maintenance
0,35.247143,18.106448,3.190000,1.990609,87.624286,101.07,0
1,51.551176,19.602963,4.645294,2.929431,95.776471,116.43,0
2,32.182308,23.743977,3.322308,2.747863,86.092308,111.96,0
3,48.806154,26.267518,4.930000,4.060322,94.401538,118.96,0
4,23.395714,14.581911,2.060000,1.641135,81.697857,98.80,0
...,...,...,...,...,...,...,...
685,33.880000,19.776546,4.920000,2.640334,99.690000,99.69,0
686,26.100000,20.123219,2.770000,3.139501,108.547500,122.05,1
687,26.500000,14.064275,2.782500,1.863635,107.690000,116.41,1
688,34.433333,22.585679,5.708333,4.833276,112.715833,136.30,1
